In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import time
import threading
from datetime import datetime, timedelta

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import OrderId, TickerId

warnings.filterwarnings('ignore')

class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        EWrapper.__init__(self)
        self.data = []
        self.contract_details_received = False
        self.historical_data_received = False
        self.next_order_id = None
        self.contract = None

    def nextValidId(self, orderId: OrderId):
        """Callback when connection is established"""
        super().nextValidId(orderId)
        self.next_order_id = orderId
        print(f"Next valid order ID: {orderId}")

    def error(self, reqId: TickerId, errorCode: int, errorString: str, advancedOrderRejectJson=""):
        """Error handling"""
        print(f"Error {errorCode}: {errorString}")

    def contractDetails(self, reqId: int, contractDetails):
        """Callback for contract details"""
        self.contract = contractDetails.contract
        self.contract_details_received = True
        print(f"Contract details received: {contractDetails.contract.symbol}")

    def historicalData(self, reqId: TickerId, bar):
        """Callback for historical data bars"""
        self.data.append({
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume
        })

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        """Callback when historical data is complete"""
        self.historical_data_received = True
        print(f"Historical data received: {len(self.data)} bars")

    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderState):
        """Callback for open orders"""
        print(f"Open Order - ID: {orderId}, Symbol: {contract.symbol}, "
              f"Action: {order.action}, Quantity: {order.totalQuantity}, "
              f"Price: {order.lmtPrice}")

    def orderStatus(self, orderId: OrderId, status: str, filled: float,
                   remaining: float, avgFillPrice: float, permId: int,
                   parentId: int, lastFillPrice: float, clientId: int, whyHeld: str, mktCapPrice: float):
        """Callback for order status updates"""
        print(f"Order Status - ID: {orderId}, Status: {status}, "
              f"Filled: {filled}, Remaining: {remaining}, Avg Price: {avgFillPrice}")

class GBPUSDTradingBot:
    def __init__(self):
        self.app = IBApi()
        self.df = None
        self.model = None
        self.forecast = None
        self.current_price = None

    def connect_to_ib(self, host="127.0.0.1", port=7497, client_id=1):
        """Connect to Interactive Brokers TWS or IB Gateway"""
        try:
            self.app.connect(host, port, client_id)
            # Start the socket in a thread
            api_thread = threading.Thread(target=self.app.run, daemon=True)
            api_thread.start()

            # Wait for connection
            time.sleep(2)
            print("Connected to Interactive Brokers")
            return True
        except Exception as e:
            print(f"Failed to connect to IB: {e}")
            return False

    def get_contract_details(self):
        """Get GBP/USD contract details"""
        contract = Contract()
        contract.symbol = "GBP"
        contract.secType = "CASH"
        contract.exchange = "IDEALPRO"
        contract.currency = "USD"

        self.app.reqContractDetails(1, contract)

        # Wait for contract details
        timeout = 10
        while not self.app.contract_details_received and timeout > 0:
            time.sleep(0.5)
            timeout -= 0.5

        if not self.app.contract_details_received:
            print("Failed to receive contract details")
            return False
        return True

    def fetch_historical_data(self, duration="1 Y", bar_size="1 day"):
        """Fetch historical GBP/USD data"""
        if not self.app.contract:
            print("No contract available. Get contract details first.")
            return False

        # Clear previous data
        self.app.data = []
        self.app.historical_data_received = False

        # Request historical data with proper timezone format
        # Use empty string for endDateTime to get most recent data
        end_date = ""  # Empty string gets most recent data

        print(f"Requesting historical data: {duration}, {bar_size}")

        self.app.reqHistoricalData(
            reqId=2,
            contract=self.app.contract,
            endDateTime=end_date,  # Empty string for most recent
            durationStr=duration,
            barSizeSetting=bar_size,
            whatToShow="MIDPOINT",
            useRTH=1,  # Regular trading hours only
            formatDate=1,  # String format for dates
            keepUpToDate=False,
            chartOptions=[]
        )

        # Wait for data with longer timeout
        timeout = 60  # Increased timeout
        print("Waiting for historical data...")
        while not self.app.historical_data_received and timeout > 0:
            time.sleep(1)
            timeout -= 1
            if timeout % 10 == 0:
                print(f"Still waiting... {timeout}s remaining")

        if not self.app.historical_data_received:
            print("Failed to receive historical data - timeout")
            print(f"Received {len(self.app.data)} data points so far")
            if len(self.app.data) == 0:
                return False
            # Continue with partial data if we have some

        # Convert to DataFrame
        if len(self.app.data) == 0:
            print("No data received")
            return False

        self.df = pd.DataFrame(self.app.data)

        # Handle date parsing more robustly
        try:
            self.df['date'] = pd.to_datetime(self.df['date'])
        except:
            # If date parsing fails, try different format
            print("Date parsing issue, attempting alternative format")
            return False

        self.df = self.df.set_index('date')
        self.df = self.df.sort_index()

        print(f"Retrieved {len(self.df)} data points")
        print(f"Date range: {self.df.index[0]} to {self.df.index[-1]}")
        return True

    def prepare_data(self, test_size=0.2):
        """Prepare data for ARIMA modeling"""
        if self.df is None or self.df.empty:
            print("No data available. Fetch historical data first.")
            return False

        # Use closing prices for modeling
        prices = self.df['close'].dropna()

        # Split data
        split_point = int(len(prices) * (1 - test_size))
        self.train_data = prices[:split_point]
        self.test_data = prices[split_point:]

        print(f"Training data: {len(self.train_data)} points")
        print(f"Testing data: {len(self.test_data)} points")

        return True

    def check_stationarity(self, data):
        """Check if the time series is stationary"""
        result = adfuller(data.dropna())
        print(f"ADF Statistic: {result[0]:.6f}")
        print(f"p-value: {result[1]:.6f}")

        if result[1] <= 0.05:
            print("Series is stationary")
            return True
        else:
            print("Series is not stationary")
            return False

    def find_arima_order(self, data, max_p=5, max_d=2, max_q=5):
        """Find optimal ARIMA parameters using AIC"""
        best_aic = np.inf
        best_order = None

        print("Searching for optimal ARIMA parameters...")

        for p in range(max_p + 1):
            for d in range(max_d + 1):
                for q in range(max_q + 1):
                    try:
                        model = ARIMA(data, order=(p, d, q))
                        fitted_model = model.fit()
                        aic = fitted_model.aic

                        if aic < best_aic:
                            best_aic = aic
                            best_order = (p, d, q)

                    except Exception as e:
                        continue

        print(f"Best ARIMA order: {best_order} with AIC: {best_aic:.2f}")
        return best_order

    def fit_arima_model(self, order=None):
        """Fit ARIMA model to training data"""
        if self.train_data is None:
            print("No training data available")
            return False

        # Check stationarity
        if not self.check_stationarity(self.train_data):
            print("Data may need differencing")

        # Find optimal order if not provided
        if order is None:
            order = self.find_arima_order(self.train_data)

        if order is None:
            print("Failed to find optimal ARIMA parameters")
            return False

        try:
            # Fit the model
            print(f"Fitting ARIMA{order} model...")
            self.model = ARIMA(self.train_data, order=order)
            self.fitted_model = self.model.fit()

            print("Model Summary:")
            print(self.fitted_model.summary())

            return True

        except Exception as e:
            print(f"Failed to fit ARIMA model: {e}")
            return False

    def evaluate_model(self):
        """Evaluate model performance on test data"""
        if self.fitted_model is None:
            print("No fitted model available")
            return False

        try:
            # Make predictions on test data
            forecast_steps = len(self.test_data)
            forecast_result = self.fitted_model.forecast(steps=forecast_steps)
            self.test_predictions = forecast_result

            # Calculate metrics
            mse = mean_squared_error(self.test_data, self.test_predictions)
            mae = mean_absolute_error(self.test_data, self.test_predictions)
            rmse = np.sqrt(mse)

            print(f"\nModel Performance:")
            print(f"MSE: {mse:.6f}")
            print(f"MAE: {mae:.6f}")
            print(f"RMSE: {rmse:.6f}")

            # Calculate percentage error
            mape = np.mean(np.abs((self.test_data - self.test_predictions) / self.test_data)) * 100
            print(f"MAPE: {mape:.2f}%")

            return True

        except Exception as e:
            print(f"Failed to evaluate model: {e}")
            return False

    def plot_results(self):
        """Plot actual vs predicted values"""
        if self.test_predictions is None:
            print("No predictions available")
            return

        plt.figure(figsize=(15, 10))

        # Plot 1: Training and test data with predictions
        plt.subplot(2, 1, 1)
        plt.plot(self.train_data.index, self.train_data, label='Training Data', alpha=0.7)
        plt.plot(self.test_data.index, self.test_data, label='Actual Test Data', color='blue')
        plt.plot(self.test_data.index, self.test_predictions, label='Predictions', color='red', linestyle='--')
        plt.title('GBP/USD Price Prediction using ARIMA')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Plot 2: Residuals
        plt.subplot(2, 1, 2)
        residuals = self.test_data - self.test_predictions
        plt.plot(self.test_data.index, residuals, label='Residuals', color='green')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        plt.title('Prediction Residuals')
        plt.xlabel('Date')
        plt.ylabel('Residual')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def predict_future(self, steps=5):
        """Predict future values"""
        if self.fitted_model is None:
            print("No fitted model available")
            return None

        try:
            # Make future predictions
            self.forecast = self.fitted_model.forecast(steps=steps)
            forecast_conf_int = self.fitted_model.forecast(steps=steps, alpha=0.05)  # 95% confidence interval

            print(f"\nFuture Predictions (next {steps} periods):")
            for i, pred in enumerate(self.forecast, 1):
                print(f"Period {i}: {pred:.4f}")

            return self.forecast

        except Exception as e:
            print(f"Failed to make future predictions: {e}")
            return None

    def create_order(self, action, quantity, limit_price):
        """Create a limit order"""
        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = limit_price
        order.tif = "GTC"  # Good Till Cancelled
        order.eTradeOnly = False
        order.firmQuoteOnly = False

        return order

    def place_trade_based_on_prediction(self, quantity=10000, confidence_threshold=0.001):
        """Place trade based on ARIMA model prediction"""
        if self.forecast is None:
            print("No forecast available. Run predict_future() first.")
            return False

        if not self.app.contract or not self.app.next_order_id:
            print("No contract or order ID available")
            return False

        try:
            # Get current price (last close price)
            current_price = self.df['close'].iloc[-1]
            predicted_price = self.forecast[0]  # Next period prediction

            price_change = predicted_price - current_price
            price_change_pct = (price_change / current_price) * 100

            print(f"\nTrading Decision Analysis:")
            print(f"Current Price: {current_price:.4f}")
            print(f"Predicted Price: {predicted_price:.4f}")
            print(f"Expected Change: {price_change:.4f} ({price_change_pct:.2f}%)")

            # Decision logic
            if abs(price_change_pct) < confidence_threshold * 100:
                print("Price change too small, no trade placed")
                return False

            # Determine action
            if predicted_price > current_price:
                action = "BUY"
                # Place buy order slightly below current price
                limit_price = current_price * 0.999  # 0.1% below current price
            else:
                action = "SELL"
                # Place sell order slightly above current price
                limit_price = current_price * 1.001  # 0.1% above current price

            # Create and place order
            order = self.create_order(action, quantity, limit_price)

            print(f"\nPlacing {action} order:")
            print(f"Quantity: {quantity}")
            print(f"Limit Price: {limit_price:.4f}")

            self.app.placeOrder(self.app.next_order_id, self.app.contract, order)
            self.app.next_order_id += 1

            print("Order placed successfully!")
            return True

        except Exception as e:
            print(f"Failed to place trade: {e}")
            return False

    def create_sample_data(self, days=365):
        """Create sample GBP/USD data for testing if IB data unavailable"""
        print(f"Creating sample GBP/USD data for {days} days...")

        # Create date range
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        dates = pd.date_range(start=start_date, end=end_date, freq='D')

        # Generate realistic GBP/USD prices using random walk
        np.random.seed(42)  # For reproducible results
        initial_price = 1.2500  # Starting around typical GBP/USD level

        # Generate price changes
        returns = np.random.normal(0, 0.005, len(dates))  # 0.5% daily volatility

        # Create cumulative prices
        prices = [initial_price]
        for ret in returns[1:]:
            prices.append(prices[-1] * (1 + ret))

        # Create DataFrame
        self.df = pd.DataFrame({
            'open': prices,
            'high': [p * (1 + abs(np.random.normal(0, 0.002))) for p in prices],
            'low': [p * (1 - abs(np.random.normal(0, 0.002))) for p in prices],
            'close': prices,
            'volume': np.random.randint(10000, 100000, len(dates))
        }, index=dates)

        print(f"Sample data created: {len(self.df)} points")
        print(f"Price range: {self.df['close'].min():.4f} - {self.df['close'].max():.4f}")

        return True

    def run_with_sample_data(self):
        """Run analysis with sample data (for testing without IB connection)"""
        print("=== Running with Sample Data (Testing Mode) ===")

        print("1. Creating sample GBP/USD data...")
        if not self.create_sample_data():
            return False

        print("2. Preparing data...")
        if not self.prepare_data():
            return False

        print("3. Fitting ARIMA model...")
        if not self.fit_arima_model():
            return False

        print("4. Evaluating model...")
        if not self.evaluate_model():
            return False

        print("5. Plotting results...")
        try:
            self.plot_results()
        except Exception as e:
            print(f"Plotting failed: {e}")

        print("6. Making future predictions...")
        self.predict_future(steps=5)

        print("\nSample analysis complete!")
        print("Note: This used sample data. Connect to IB for real trading.")
        return True

    def run_complete_analysis(self):
        """Run the complete analysis pipeline"""
        print("=== GBP/USD ARIMA Trading Bot ===")
        print("1. Connecting to Interactive Brokers...")

        if not self.connect_to_ib():
            return False

        print("2. Getting contract details...")
        if not self.get_contract_details():
            return False

        print("3. Fetching historical data...")
        # Try different data requests if first one fails
        success = False
        data_configs = [
            ("1 Y", "1 day"),
            ("6 M", "1 day"),
            ("3 M", "1 day"),
            ("1 M", "1 day"),
            ("2 W", "1 hour")
        ]

        for duration, bar_size in data_configs:
            print(f"Trying: {duration}, {bar_size}")
            if self.fetch_historical_data(duration, bar_size):
                success = True
                break
            time.sleep(2)  # Wait between attempts

        if not success:
            print("Failed to fetch any historical data")
            return False

        print("4. Preparing data...")
        if not self.prepare_data():
            return False

        print("5. Fitting ARIMA model...")
        if not self.fit_arima_model():
            return False

        print("6. Evaluating model...")
        if not self.evaluate_model():
            return False

        print("7. Plotting results...")
        try:
            self.plot_results()
        except Exception as e:
            print(f"Plotting failed: {e}")

        print("8. Making future predictions...")
        self.predict_future(steps=5)

        print("9. Placing trade based on prediction...")
        try:
            self.place_trade_based_on_prediction()
        except Exception as e:
            print(f"Trade placement failed: {e}")

        print("\nAnalysis complete!")
        return True
        """Run the complete analysis pipeline"""
        print("=== GBP/USD ARIMA Trading Bot ===")
        print("1. Connecting to Interactive Brokers...")

        if not self.connect_to_ib():
            return False

        print("2. Getting contract details...")
        if not self.get_contract_details():
            return False

        print("3. Fetching historical data...")
        # Try different data requests if first one fails
        success = False
        data_configs = [
            ("1 Y", "1 day"),
            ("6 M", "1 day"),
            ("3 M", "1 day"),
            ("1 M", "1 day"),
            ("2 W", "1 hour")
        ]

        for duration, bar_size in data_configs:
            print(f"Trying: {duration}, {bar_size}")
            if self.fetch_historical_data(duration, bar_size):
                success = True
                break
            time.sleep(2)  # Wait between attempts

        if not success:
            print("Failed to fetch any historical data")
            return False

        print("4. Preparing data...")
        if not self.prepare_data():
            return False

        print("5. Fitting ARIMA model...")
        if not self.fit_arima_model():
            return False

        print("6. Evaluating model...")
        if not self.evaluate_model():
            return False

        print("7. Plotting results...")
        try:
            self.plot_results()
        except Exception as e:
            print(f"Plotting failed: {e}")

        print("8. Making future predictions...")
        self.predict_future(steps=5)

        print("9. Placing trade based on prediction...")
        try:
            self.place_trade_based_on_prediction()
        except Exception as e:
            print(f"Trade placement failed: {e}")

        print("\nAnalysis complete!")
        return True

# Usage example
if __name__ == "__main__":
    # Create trading bot instance
    bot = GBPUSDTradingBot()

    # Option 1: Try with Interactive Brokers connection
    print("Attempting to run with Interactive Brokers connection...")
    success = bot.run_complete_analysis()

    # Option 2: If IB fails, run with sample data for testing
    if not success:
        print("\n" + "="*50)
        print("IB connection failed. Running with sample data...")
        print("="*50)
        bot = GBPUSDTradingBot()  # Create fresh instance
        bot.run_with_sample_data()
    else:
        # Keep the connection alive for a while to see order status
        print("\nMonitoring orders for 30 seconds...")
        time.sleep(30)

        # Disconnect
        bot.app.disconnect()

Attempting to run with Interactive Brokers connection...
=== GBP/USD ARIMA Trading Bot ===
1. Connecting to Interactive Brokers...
Next valid order ID: 1
Error 2104: Market data farm connection is OK:usfarm.nj
Error 2104: Market data farm connection is OK:usfuture
Error 2104: Market data farm connection is OK:cashfarm
Error 2104: Market data farm connection is OK:usfarm
Error 2106: HMDS data farm connection is OK:ushmds
Error 2158: Sec-def data farm connection is OK:secdefil
Connected to Interactive Brokers
2. Getting contract details...
Contract details received: GBP
3. Fetching historical data...
Trying: 1 Y, 1 day
Requesting historical data: 1 Y, 1 day
Waiting for historical data...
Error 10285: Your API version does not support fractional size rules. Please upgrade to a minimum version 163.
Still waiting... 50s remaining


KeyboardInterrupt: 